In [7]:
import dask.dataframe as dd
gct = dd.read_csv("../data/gtex/GTEx_Analysis_v6_RNA-seq_RNA-SeQCv1.1.8_gene_rpkm.gct", sep="\t", skiprows=2)
gct.head()

TypeError: Runtime type mismatch. Add {'GTEX-111CU-1826-SM-5GZYN': float} to dtype= keyword in read_csv/read_table

Traceback
---------
  File "/pstore/home/sturmg/.local/lib/python3.5/site-packages/dask/async.py", line 268, in execute_task
    result = _execute_task(task, data)
  File "/pstore/home/sturmg/.local/lib/python3.5/site-packages/dask/async.py", line 249, in _execute_task
    return func(*args2)
  File "/pstore/home/sturmg/.local/lib/python3.5/site-packages/dask/compatibility.py", line 47, in apply
    return func(*args, **kwargs)
  File "/pstore/home/sturmg/.local/lib/python3.5/site-packages/dask/dataframe/io/csv.py", line 55, in pandas_read_text
    coerce_dtypes(df, dtypes)
  File "/pstore/home/sturmg/.local/lib/python3.5/site-packages/dask/dataframe/io/csv.py", line 82, in coerce_dtypes
    raise TypeError(msg % c)


In [1]:
from pygenesig.validation import cross_validate_signatures
from pygenesig.gini import GiniSignatureGenerator
from pygenesig.bioqc import BioQCSignatureTester

expr_file = "../data/gtex/exprs.npy"
target_file = "../data/gtex/target.csv"

signature_list, result_list = cross_validate_signatures(expr_file,
                                                        target_file,
                                                        GiniSignatureGenerator,
                                                        BioQCSignatureTester)

In [3]:
import dask.multiprocessing

In [2]:
import dask.multiprocessing
from dask.base import compute

signature_futures, result_futures = compute(signature_list, result_list,
                                            get=dask.multiprocessing.get)

signatures = [f.result() for f in signature_futures]
confusion_matrices = [f.result() for f in result_futures]

AttributeError: module 'dask' has no attribute 'multiprocessing'

In [13]:
?compute

In [1]:
import readline
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri
from pygenesig.validation import SignatureTester
from pygenesig.tools import write_gmt
import sklearn.metrics
import numpy as np
from pygenesig.validation import SignatureGenerator
import tempfile
numpy2ri.activate()
import pandas as pd
from rpy2.robjects import pandas2ri, Formula
pandas2ri.activate()
import rpy2.robjects
from IPython.display import display

base = importr("base")
stats = importr("stats")
edge_r = importr("edgeR")
limma = importr("limma")

In [2]:
expr = np.load("../data/gtex/exprs_counts.npy")

In [3]:
target = np.genfromtxt("../data/gtex/target.csv", dtype=str, delimiter=",")

In [4]:
covariates = pd.read_csv("../data/gtex/covariates.csv", index_col=0)

In [5]:
genes = np.array([str(i) for i in range(expr.shape[0])])

In [6]:
expr.shape

(56318, 8525)

In [7]:
genes.shape

(56318,)

In [8]:
dg_list = edge_r.DGEList(counts=expr, genes=genes)

In [101]:
len(set(covariates.SMTS))

27

In [10]:
df = base.data_frame(covariates)

In [105]:
fmla = Formula("~ x1 + x2 + x3")
env = fmla.environment
env['x1'] = base.as_factor(covariates.SMTS)
env['x2'] = covariates.RIN
env['x3'] = base.as_factor(covariates.Gender)

In [100]:
base.length(base.levels(base.as_factor(covariates.SMTS)))

R object with classes: ('integer',) mapped to:
<IntVector - Python:0x2aae19ec82c8 / R:0x269b44f8>
[      27]

In [108]:
cpm = edge_r.cpm(dg_list)

In [109]:
cpm_cnt = np.sum(np.array(cpm) > 1, axis=1)

In [110]:
cpm_inds = cpm_cnt >= 10

In [111]:
expr_fil = expr[cpm_inds, :]
genes_fil = genes[cpm_inds]

In [112]:
dg_list_fil = edge_r.DGEList(counts=expr_fil, genes=genes_fil)

In [113]:
dg_list_fil2 = edge_r.calcNormFactors(dg_list_fil)

In [114]:
voom_res = limma.voom(dg_list_fil2, design_mat, plot=False)

In [115]:
lm_fit = limma.lmFit(voom_res, design_mat)

In [116]:
fit_e = limma.treat(lm_fit, lfc=np.log2(100))

In [117]:
p_adj = .01 / base.nrow(lm_fit)[0]

In [118]:
rslt = limma.decideTests(fit_e, p_value=p_adj)

In [119]:
colnames = np.array(base.colnames(rslt))

In [120]:
colnames

array(['(Intercept)', 'x1Adrenal Gland', 'x1Blood', 'x1Blood Vessel',
       'x1Brain', 'x1Breast', 'x1Colon', 'x1Esophagus', 'x1Heart',
       'x1Kidney', 'x1Liver', 'x1Lung', 'x1Muscle', 'x1Nerve', 'x1Ovary',
       'x1Pancreas', 'x1Pituitary', 'x1Prostate', 'x1Salivary Gland',
       'x1Skin', 'x1Small Intestine', 'x1Spleen', 'x1Stomach', 'x1Testis',
       'x1Thyroid', 'x1Uterus', 'x1Vagina', 'x2', 'x3Male'], 
      dtype='<U17')

In [121]:
np_rslt = np.array(rslt)

In [122]:
tissue_cols = np.array([True if col.startswith('x1') else False for col in colnames])
np_rslt = np_rslt[:, tissue_cols]

In [123]:
tissues = np.array([col[2:] for col in colnames[tissue_cols]])
assert len(tissues) == np_rslt.shape[1]

Translate back to row numbers before filtering

In [124]:
row_inds = np.where(cpm_inds)[0]

In [125]:
signatures = {}
for i, tissue in enumerate(tissues):
    signatures[tissue] = list(row_inds[np_rslt[:, i] == 1])    

In [80]:
signatures

{'Adrenal Gland': [9,
  34,
  35,
  36,
  92,
  94,
  99,
  102,
  122,
  133,
  134,
  148,
  165,
  181,
  217,
  231,
  254,
  263,
  274,
  284,
  300,
  314,
  322,
  340,
  347,
  392,
  399,
  489,
  500,
  509,
  569,
  576,
  593,
  594,
  606,
  622,
  640,
  646,
  655,
  666,
  673,
  724,
  730,
  768,
  780,
  838,
  877,
  878,
  883,
  887,
  953,
  973,
  1000,
  1005,
  1012,
  1036,
  1068,
  1091,
  1101,
  1131,
  1138,
  1144,
  1202,
  1233,
  1299,
  1321,
  1332,
  1373,
  1418,
  1438,
  1444,
  1462,
  1587,
  1597,
  1598,
  1633,
  1688,
  1717,
  1741,
  1887,
  1898,
  2005,
  2075,
  2122,
  2162,
  2187,
  2228,
  2246,
  2273,
  2299,
  2347,
  2369,
  2398,
  2459,
  2529,
  2555,
  2557,
  2603,
  2631,
  2712,
  2815,
  3022,
  3029,
  3033,
  3035,
  3041,
  3077,
  3079,
  3112,
  3114,
  3123,
  3124,
  3145,
  3173,
  3177,
  3178,
  3188,
  3190,
  3201,
  3205,
  3209,
  3215,
  3216,
  3223,
  3230,
  3250,
  3252,
  3257,
  3262,
  3274,
  3

In [126]:
{
    tissue: len(genes) for tissue, genes in signatures.items()
}

{'Adrenal Gland': 83,
 'Blood': 61,
 'Blood Vessel': 3,
 'Brain': 464,
 'Breast': 0,
 'Colon': 24,
 'Esophagus': 3,
 'Heart': 144,
 'Kidney': 166,
 'Liver': 432,
 'Lung': 63,
 'Muscle': 237,
 'Nerve': 32,
 'Ovary': 40,
 'Pancreas': 265,
 'Pituitary': 264,
 'Prostate': 82,
 'Salivary Gland': 179,
 'Skin': 163,
 'Small Intestine': 240,
 'Spleen': 146,
 'Stomach': 74,
 'Testis': 1871,
 'Thyroid': 100,
 'Uterus': 14,
 'Vagina': 53}

In [ ]:
stats.model_m